# Project Name
By Alec Plante ...

### Import Libraries


In [1]:
import pandas as pd
import numpy as np
import sqlite3


### Unzip Data
This section is used to unzip data from the zippedData folder and place it into the new data folder

In [2]:
#extract im.db zip file
import zipfile
with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
    zip_ref.extractall('data/')

# unzip the gz files 
import gzip
import shutil

# unzip bom.movie_gross
with gzip.open('zippedData/bom.movie_gross.csv.gz', 'rb') as f_in:
    with open('data/bom.movie_gross.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
# unzip rt.movie_info.tsv
with gzip.open('zippedData/rt.movie_info.tsv.gz', 'rb') as f_in:
    with open('data/rt.movie_info.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
# unzip rt.reviews.tsv
with gzip.open('zippedData/rt.reviews.tsv.gz', 'rb') as f_in:
    with open('data/rt.reviews.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
# unzip tmdb.movies.csv
with gzip.open('zippedData/tmdb.movies.csv.gz', 'rb') as f_in:
    with open('data/tmdb.movies.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
# unzip tn.movie_budgets.csv
with gzip.open('zippedData/tn.movie_budgets.csv.gz', 'rb') as f_in:
    with open('data/tn.movie_budgets.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

### Import Data and connect to Database

In [4]:
# import data as 
movieGross = pd.read_csv('data/bom.movie_gross.csv')
tmdbMovies = pd.read_csv('data/tmdb.movies.csv')
movieBudgets = pd.read_csv('data/tn.movie_budgets.csv')
movieInfo = pd.read_csv('data/rt.movie_info.tsv', sep = '\t')
reviews = pd.read_csv('data/rt.reviews.tsv', sep = '\t', encoding = 'latin1')


In [5]:
# Connect to sql database
conn = sqlite3.connect('data/im.db')

In [ ]:
# Converts foreign_gross column to string and removes commas
movieGross['foreign_gross'] = movieGross['foreign_gross'].astype(str).str.replace(\",\",\"\")\n",
# Converts Null values in foreign_gross column to 0\n"
movieGross['foreign_gross'] = movieGross['foreign_gross'].fillna(0)
# Converts foreign_gross column from object type\n",
    "movieGross['foreign_gross'] = movieGross['foreign_gross'].astype(float).astype(int)\n",
    "\n",
    "# Converts domestic_gross column values to integers and Null values in domestic_gross column to 0 \n",
    "movieGross['domestic_gross'] = movieGross['domestic_gross'].fillna(0).astype(int)\n",
    "\n",
    "# Converts year column to datetime data type\n",
    "movieGross['year'] = pd.to_datetime(movieGross['year'],format = '%Y')\n",
    "\n",
    "movieGross"
   "source": [
    "Questions:\n",
    "- What is the growth of popularity by genre?\n",
    "- Which genres are the most profitable?\n",
    "- Whats the relationship between run time and profitability?\n",
    "- Which directors have the most popular movies\n"
